In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
df = pd.read_excel('data_1000_samples_clean.xlsx')
df

In [ ]:
train_df = df[['Barometer']][200:248]
val_df = df[['Barometer']][248:296]
test_df = df[['Barometer']][248:296]

In [ ]:
# train_max = train_df.max()
# val_max = val_df.max()
# test_max = test_df.max()
# max_ = 0
# if train_max.Barometer > val_max.Barometer:
#     max_ = train_max
# else:
#     max_ = val_max
    
# train_df = train_df/ max_
# val_df = val_df/ max_
# test_df = test_df/ max_

### Feature Engineering: Window Features

In [ ]:
class WindowGenerator():
    def __init__(self, input_width, label_width, shift,
                 train_df, val_df, test_df,
                 label_columns=None):

        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df

        self.label_columns = label_columns # = ['Temp','Wind','Humidity','Barometer']
        if label_columns is not None:
            self.label_columns_indices = {name: i for i, name in enumerate(label_columns)} #chỉ số label
        #{'Temp': 0, 'Wind': 1, 'Humidity': 2, 'Barometer': 3}
        
        self.column_indices = {name: i for i, name in enumerate(train_df.columns)} 
        #{'Temp': 0, 'Wind': 1, 'Humidity': 2, 'Barometer': 3}

        self.input_width = input_width # = 24
        self.label_width = label_width # = 24
        self.shift = shift # = 24

        self.total_window_size = input_width + shift # = 48

        self.input_slice = slice(0, input_width) # slice(0, 24)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice] # np.arange(48)[slice(0, 24)]

        self.label_start = self.total_window_size - self.label_width # = 24
        self.labels_slice = slice(self.label_start, None) # slice(24, 48)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice] # np.arange(48)[slice(24, 48)]

    def __repr__(self):
        return '\n'.join([
            f'Total window size: {self.total_window_size}',
            f'Input indices: {self.input_indices}',
            f'Label indices: {self.label_indices}',
            f'Label column name(s): {self.label_columns}'
        ])

    def train(self):
        return self.make_dataset(self.train_df)

    def val(self):
        return self.make_dataset(self.val_df)

    def test(self):
        return self.make_dataset(self.test_df)

In [ ]:
# Hàm chia input và output
def split_window(self, features):
    #Chia dữ liệu thành 2 inputs và labels
    inputs = features[:, self.input_slice, :] #input_slice = slice(0, 24)
    labels = features[:, self.labels_slice, :] #labels_slice = slice(24, 48)
    
    #Lấy dữ liệu dựa trên label
    if self.label_columns is not None:
        # label_columns = ['Temp','Wind','Humidity','Barometer']
        # column_indices[Temp] = 0 ; column_indices[Wind] = 1;... 
        labels = tf.stack( [labels[:, :, self.column_indices[name]] for name in self.label_columns], axis=-1)

    # Slicing doesn't preserve static shape information, so set the shapes
    # manually. This way the `tf.data.Datasets` are easier to inspect.
    inputs.set_shape([None, self.input_width, None])
    labels.set_shape([None, self.label_width, None])

    return inputs, labels

WindowGenerator.split_window = split_window


In [ ]:

def make_dataset(self, data):
    data = np.array(data, dtype=np.float32) # chuyển dataframe về numpy array
    ds = tf.keras.utils.timeseries_dataset_from_array(
      data=data,
      targets=None,
      sequence_length=self.total_window_size,
      sequence_stride=1,
      shuffle=False,
      batch_size=32,)
    ds = ds.map(self.split_window)

    return ds

WindowGenerator.make_dataset = make_dataset

In [ ]:
@property
def train(self):
    return self.make_dataset(self.train_df)

@property
def val(self):
    return self.make_dataset(self.val_df)

@property
def test(self):
    return self.make_dataset(self.test_df)

@property
def example(self):
    """Get and cache an example batch of `inputs, labels` for plotting."""
    result = getattr(self, '_example', None)
    if result is None:
        # No example batch was found, so get one from the `.train` dataset
        result = next(iter(self.test))
        # And cache it for next time
        self._example = result
    return result

In [ ]:
WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example

In [ ]:
OUT_STEPS = 24
multi_window = WindowGenerator(input_width=24,
                               label_width=OUT_STEPS,
                               shift=OUT_STEPS,
                              train_df = train_df,
                              val_df = val_df,
                              test_df = test_df,
                              label_columns = ['Barometer'])

In [ ]:
multi_window.__repr__()

## Training

In [ ]:
import tensorflow as tf
from sklearn.metrics import r2_score
MAX_EPOCHS = 200

def mean_2(y_true, y_pred):
    MAE = tf.reduce_mean(tf.abs(y_true - y_pred))

    y_sub = tf.subtract(y_true[:, 1:, :], y_true[:, :-1, :])
    y_hat_sub = tf.subtract(y_pred[:, 1:, :], y_pred[:, :-1, :])

    n = y_sub.shape[1] + y_sub.shape[2]
    
    A = y_sub*y_hat_sub + 1
    B = tf.sqrt(tf.square(y_sub) + 1)*tf.sqrt(tf.square(y_hat_sub) + 1)
    
    alpha = 1
    
    vector_similarity = tf.divide(A,B)
    sum_VS = tf.reduce_mean(vector_similarity)
    sum_VS = sum_VS*alpha
    
    MSE_2 = MAE + tf.math.reduce_std(tf.abs(y_true - y_pred))/MAE - sum_VS 
    
    return MSE_2
def CV(y_true, y_pred):
    MAE = tf.reduce_mean(tf.abs(y_true - y_pred))
    MSE_2 = tf.math.reduce_std(tf.abs(y_true - y_pred))/MAE 
    
    return MSE_2

def compile_and_fit(model, window, patience=2):
    model.compile(loss=tf.losses.MeanSquaredError(),
                  optimizer=tf.optimizers.Adam(),
                  metrics=[CV,mean_2,tf.metrics.MeanAbsoluteError()])
    history = model.fit(window.train, epochs=MAX_EPOCHS,
                        validation_data=window.val)
    return history


In [ ]:
class MultiStepLastBaseline(tf.keras.Model):
    def call(self, inputs):
        return tf.tile(inputs[:, -1:, :], [1, OUT_STEPS, 1])

num_features = train_df.shape[1]
CONV_WIDTH = 3
multi_conv_model = MultiStepLastBaseline()

In [ ]:
history = compile_and_fit(multi_conv_model, multi_window)

In [ ]:
multi_conv_model.summary()

## Trực quan dữ liệu sau khi Training

In [ ]:
def plot(self, model=None, plot_col='Temp', max_subplots=3):
    inputs, labels = self.example
    plt.figure(figsize=(19, 3))
    plot_col_index = self.column_indices[plot_col]
    max_n = min(max_subplots, len(inputs))
    for n in range(max_n):
        plt.subplot(max_n, 1, n+1)
        plt.ylabel(f'{plot_col}')
        plt.plot(self.input_indices, inputs[n, :, plot_col_index],
                 label='Inputs', marker='.', zorder=-10)

        if self.label_columns:
            label_col_index = self.label_columns_indices.get(plot_col, None)
        else:
            label_col_index = plot_col_index

        if label_col_index is None:
            continue

        plt.scatter(self.label_indices, labels[n, :, label_col_index],
                    edgecolors='k', label='Labels', c='#2ca02c', s=64)
        if model is not None:
            predictions = model(inputs)
            plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                        marker='X', edgecolors='k', label='Predictions',
                        c='#ff7f0e', s=64)

        if n == 0:
            plt.legend()

    plt.xlabel('Time ')

WindowGenerator.plot = plot


### Độ chính xác cao hơn mô hình cũ

In [ ]:
multi_window.plot(multi_conv_model,plot_col='Barometer')

In [ ]:
import matplotlib.pyplot as plt
# Vẽ đồ thị
# Chỉnh kích thước đồ thị
plt.plot(history.history['val_CV'], label='CV')
plt.plot(history.history['val_mean_2'], label='New Metric')
plt.plot(history.history['val_mean_absolute_error'], label='MAE')

# Đặt tên trục x và trục y
plt.xlabel('epochs')
plt.ylabel('loss')

# Thêm chú thích
plt.legend()
plt.figure(figsize=(8, 13))
# Hiển thị đồ thị
plt.show()

In [ ]:
inputs, labels = multi_window.example
predict = np.array(multi_conv_model(inputs)).reshape(24,)
labels = np.array(labels).reshape(24,)
labels.shape

In [ ]:
from scipy.stats import ks_2samp
# Thực hiện kiểm định Kolmogorov-Smirnov
statistic, p_value = ks_2samp(predict, labels)

# In kết quả
print("Statistic:", statistic)
print("p-value:", p_value)

In [ ]:
predict.shape